In [66]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

In [67]:
import pandas as pd

In [68]:
import numpy as np

In [89]:
trainx = pd.read_csv('C:/Users/Prasanth/Desktop/Allstate/train.csv')

In [70]:
test = pd.read_csv('C:/Users/Prasanth/Desktop/Allstate/test.csv')

In [90]:
trainy = trainx.pop("loss")

In [92]:
trainy.head()

0    2213.18
1    1283.60
2    3005.09
3     939.85
4    2763.85
Name: loss, dtype: float64

In [93]:
numericvariables = list(trainx.dtypes[trainx.dtypes != 'object'].index)

In [76]:
model = RandomForestRegressor(n_estimators=100, oob_score=True, random_state=42)

In [77]:
model.fit(trainx[numericvariables], trainy)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=100, n_jobs=1, oob_score=True, random_state=42,
           verbose=0, warm_start=False)

In [78]:
model.oob_score_

-0.046010061017927262

In [79]:
model.feature_importances_

array([ 0.25371899,  0.03023805,  0.05629928,  0.02903212,  0.02815307,
        0.04163815,  0.0421068 ,  0.09894493,  0.03434581,  0.02009871,
        0.02170794,  0.02692746,  0.02301148,  0.03140887,  0.26236834])

In [80]:
feature_importance = pd.Series(model.feature_importances_, index=numericvariables)

In [81]:
feature_importance.sort_values()

cont9     0.020099
cont10    0.021708
cont12    0.023011
cont11    0.026927
cont4     0.028153
cont3     0.029032
cont1     0.030238
cont13    0.031409
cont8     0.034346
cont5     0.041638
cont6     0.042107
cont2     0.056299
cont7     0.098945
id        0.253719
cont14    0.262368
dtype: float64

In [95]:
Categorical_vars = list(trainx.dtypes[trainx.dtypes == 'object'].index)

In [97]:
for variable in Categorical_vars:
    dummies = pd.get_dummies(trainx[variable], prefix=variable)
    trainx = pd.concat([trainx, dummies], axis=1)
    trainx.drop([variable], axis = 1, inplace = True)

In [112]:
trainx.shape

(188318, 1153)

In [124]:
trainxcols = list(trainx.columns.values)

In [110]:
for variable in Categorical_vars:
    dummies = pd.get_dummies(test[variable], prefix=variable)
    test = pd.concat([test, dummies], axis=1)
    test.drop([variable], axis = 1, inplace = True)

In [113]:
test.shape

(125546, 1117)

In [115]:
testcolnames = list(test.columns.values)


In [125]:
Colsformodel =  list(set(trainxcols).intersection(testcolnames))


In [129]:
len(Colsformodel)

1079

In [154]:
%%timeit
model2 = RandomForestRegressor(n_estimators=100, oob_score=True, random_state=42, n_jobs=-1, min_samples_leaf = 5)
model2.fit(trainx[Colsformodel], trainy)

KeyboardInterrupt: 

In [132]:
model2.oob_score_

0.5300501257486524

In [133]:
model2.feature_importances_

array([  3.67871355e-05,   1.35952155e-04,   1.20998034e-03, ...,
         0.00000000e+00,   1.23979023e-03,   2.70974959e-09])

In [135]:
feature_importance = pd.Series(model2.feature_importances_, index=trainx[Colsformodel].columns)
feature_importance.sort_values()
feature_importance.plot(kind = "barh")

In [139]:
Res = model2.predict(test[Colsformodel])

In [143]:
Res

array([ 2048.7076,  2187.3143,  9400.8141, ...,  2988.9007,  1523.0604,
        3369.1904])

In [147]:
Fin = pd.DataFrame({'id':test['id'],'loss':Res})

In [148]:
Fin

,id,loss
0,4,2048.7076
1,6,2187.3143
2,9,9400.8141
3,12,6607.6074
4,15,764.8152
5,17,2252.1718
6,21,2282.6114
7,28,1162.1650
8,32,3957.3109
9,43,3137.2271


In [153]:
Fin.to_csv('RandForest.csv', sep=',', index=False)

In [156]:
trainx[Colsformodel].to_csv('TransformedTrain.csv', sep=',', index = False)